In [1]:
import pandas as pd
import numpy as np

import nltk
from nltk.tokenize import sent_tokenize
import re

In [36]:
df = pd.read_csv('data/113_gsm8k_gpt35_cot_responses.csv')

In [37]:
df.head()

,Problem Number,Question,Answer,CoT Response,CoT Sentences,CoT Steps,split,Paraphrased Question,Paraphrased CoT Steps
0,0,Natalia sold clips to 48 of her friends in Apr...,Natalia sold 48/2 = <<48/2=24>>24 clips in May...,"First, let's find out how many clips Natalia s...","**\nFirst, let's find out how many clips Natal...","[""First, let's find out how many clips Natalia...",train,Derek collected seashells from 48 of his frien...,"['First, we need to figure out the number of s..."
1,1,Weng earns $12 an hour for babysitting. Yester...,Weng earns 12/60 = $<<12/60=0.2>>0.2 per minut...,"To find out how much Weng earned, we need to c...","**\nTo find out how much Weng earned, we need ...","['To find out how much Weng earned, we need to...",train,Liam makes $12 per hour tutoring. He tutored f...,"[""First, we need to figure out Liam's earnings..."
2,2,Betty is saving money for a new wallet which c...,"In the beginning, Betty has only 100 / 2 = $<<...","First, let's calculate how much money Betty ha...","**\nFirst, let's calculate how much money Bett...","[""First, let's calculate how much money Betty ...",train,Oliver is collecting stickers for his album wh...,"['First, we need to figure out how many sticke..."
3,3,"Julie is reading a 120-page book. Yesterday, s...",Maila read 12 x 2 = <<12*2=24>>24 pages today....,"First, let's find out how many pages Julie has...","**\nFirst, let's find out how many pages Julie...","[""First, let's find out how many pages Julie h...",train,Carlos is painting a fence that has 120 panels...,"['First, we need to calculate the number of pa..."
4,4,James writes a 3-page letter to 2 different fr...,He writes each friend 3*2=<<3*2=6>>6 pages a w...,"First, let's calculate how many letters James ...","**\nFirst, let's calculate how many letters Ja...","[""First, let's calculate how many letters Jame...",train,Olivia plants 3 flowers in each of 2 different...,"['First, we need to figure out how many times ..."


In [38]:
def extract_answer(answer_str):
    match = re.search(r'####\s*([\d.]+)', answer_str)
    if match:
        number = match.group(1)
        return number
    else:
        return None
print(extract_answer(df['Answer'][0]))
df['True Answer'] = df['Answer'].apply(extract_answer)

72


In [39]:
def check_last_sentence_for_exact_answer(row):
    explanation = row['CoT Response']
    numeric_answer = str(float(row['True Answer']))

    # Splitting the explanation on new lines and then tokenize each part into sentences
    parts = explanation.split('\n')
    all_sentences = [sent for part in parts for sent in sent_tokenize(part)]
    last_sentence = all_sentences[-1] if all_sentences else ""

    # Extract all strings that might contain numeric values
    pattern = r"(?:(?<=\$)\s*)?\b\d{1,3}(?:,\d{3})*(?:\.\d+)?\b(?:\s*(?=\%))?"
    potential_numeric_strings = re.findall(pattern, last_sentence)

    for string in potential_numeric_strings:
        # Removing non-numeric characters
        cleaned_string = re.sub(r'[^\d.]', '', string)
        cleaned_string2 = str(float(cleaned_string))
        # Check if the cleaned string matches the numeric answer
        if cleaned_string2 == numeric_answer:
            return 1

    return 0

df['Score_ExactAnswerInLastSentence'] = df.apply(check_last_sentence_for_exact_answer, axis=1)

In [40]:
def extract_numerical_answer(row):
    explanation = row['CoT Response']
    numeric_answer = str(float(row['True Answer']))

    pattern = r"(?:(?<=\$)\s*)?\b\d{1,3}(?:,\d{3})*(?:\.\d+)?\b(?:\s*(?=\%))?"
    matches = re.findall(pattern, explanation)
    numbers = [float(match.replace(',', '')) if '.' in match else int(match.replace(',', '')) for match in matches if match]
    try:
        answer = numbers[-2:]
        # print(answer)
        # answer = str(float(numbers[-1]))
        for a in answer:
            if str(float(a)) == numeric_answer:
                return 1
        return 0
    except:
        print(row)
        return 2

df['Score_LastNumberInString'] = df.apply(extract_numerical_answer, axis=1)


In [41]:
df['Final Score'] = 2

In [42]:
df.head()

,Problem Number,Question,Answer,CoT Response,CoT Sentences,CoT Steps,split,Paraphrased Question,Paraphrased CoT Steps,True Answer,Score_ExactAnswerInLastSentence,Score_LastNumberInString,Final Score
0,0,Natalia sold clips to 48 of her friends in Apr...,Natalia sold 48/2 = <<48/2=24>>24 clips in May...,"First, let's find out how many clips Natalia s...","**\nFirst, let's find out how many clips Natal...","[""First, let's find out how many clips Natalia...",train,Derek collected seashells from 48 of his frien...,"['First, we need to figure out the number of s...",72,1,1,2
1,1,Weng earns $12 an hour for babysitting. Yester...,Weng earns 12/60 = $<<12/60=0.2>>0.2 per minut...,"To find out how much Weng earned, we need to c...","**\nTo find out how much Weng earned, we need ...","['To find out how much Weng earned, we need to...",train,Liam makes $12 per hour tutoring. He tutored f...,"[""First, we need to figure out Liam's earnings...",10,1,1,2
2,2,Betty is saving money for a new wallet which c...,"In the beginning, Betty has only 100 / 2 = $<<...","First, let's calculate how much money Betty ha...","**\nFirst, let's calculate how much money Bett...","[""First, let's calculate how much money Betty ...",train,Oliver is collecting stickers for his album wh...,"['First, we need to figure out how many sticke...",5,1,1,2
3,3,"Julie is reading a 120-page book. Yesterday, s...",Maila read 12 x 2 = <<12*2=24>>24 pages today....,"First, let's find out how many pages Julie has...","**\nFirst, let's find out how many pages Julie...","[""First, let's find out how many pages Julie h...",train,Carlos is painting a fence that has 120 panels...,"['First, we need to calculate the number of pa...",42,1,1,2
4,4,James writes a 3-page letter to 2 different fr...,He writes each friend 3*2=<<3*2=6>>6 pages a w...,"First, let's calculate how many letters James ...","**\nFirst, let's calculate how many letters Ja...","[""First, let's calculate how many letters Jame...",train,Olivia plants 3 flowers in each of 2 different...,"['First, we need to figure out how many times ...",624,1,1,2


In [43]:
def set_final_score(row):
    if row['Score_LastNumberInString'] == row['Score_ExactAnswerInLastSentence']:
        return row['Score_LastNumberInString']
    else:
        return row['Final Score']

def set_final_score2(row):
    if row['BenchmarkScore_LastNumberInString'] == row['BenchmarkScore_ExactAnswerInLastSentence']:
        return row['BenchmarkScore_LastNumberInString']
    else:
        return row['Benchmark Final Score']

df['Final Score'] = df.apply(set_final_score, axis=1)
# temp0['Benchmark Final Score'] = temp0.apply(set_final_score2, axis=1)


In [44]:
df['Final Score'].value_counts()

Final Score
1    80
0    32
2     1
Name: count, dtype: int64

In [25]:
c1 = df['Final Score'] == 2
human_eval = df[c1]
# human_eval.reset_index(drop=True, inplace=True)
print(len(human_eval))
human_eval.head()

1


,Problem Number,Question,Answer,CoT Response,CoT Sentences,CoT Steps,split,Paraphrased Question,Paraphrased CoT Steps,True Answer,Score_ExactAnswerInLastSentence,Score_LastNumberInString,Final Score
55,55,John writes 20 pages a day. How long will it ...,He wants to write 3*400=<<3*400=1200>>1200 pag...,"First, let's calculate how many pages John wri...","**\nFirst, let's calculate how many pages John...","[""First, let's calculate how many pages John w...",train,Ava knits 20 scarves a day. How many days will...,"['First, we need to figure out how many scarve...",60,1,0,2


In [27]:
for i, row in human_eval.iterrows():
    print('#####')
    print(i)
    print()
    print(human_eval['CoT Response'][i])
    print()
    print('In-Last Sentence: ', human_eval['Score_ExactAnswerInLastSentence'][i])
    print()
    print('Last Number in String: ', human_eval['Score_LastNumberInString'][i])
    print()
    print(human_eval['True Answer'][i])
    print()
    print('#####')

#####
55

First, let's calculate how many pages John writes in a day. Since he writes 20 pages a day, we can multiply that by 1 to get 20 pages.

Next, let's calculate how many pages are in 3 books. Since each book is 400 pages, we can multiply that by 3 to get 1200 pages.

Now, let's calculate how many days it will take John to write 1200 pages. We can divide 1200 pages by 20 pages per day to get 60 days.

Therefore, it will take John 60 days to write 3 books that are 400 pages each.

In-Last Sentence:  1

Last Number in String:  0

60

#####


In [45]:
df['Final Score'][55] = 1

/var/folders/8y/j9pbnmqj2t1bn5kv9ps9lg680000gn/T/ipykernel_9950/575722071.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Final Score'][55] = 1


In [30]:
df.head()

,Problem Number,Question,Answer,CoT Response,CoT Sentences,CoT Steps,split,Paraphrased Question,Paraphrased CoT Steps,True Answer,Score_ExactAnswerInLastSentence,Score_LastNumberInString,Final Score
0,0,Natalia sold clips to 48 of her friends in Apr...,Natalia sold 48/2 = <<48/2=24>>24 clips in May...,"First, let's find out how many clips Natalia s...","**\nFirst, let's find out how many clips Natal...","[""First, let's find out how many clips Natalia...",train,Derek collected seashells from 48 of his frien...,"['First, we need to figure out the number of s...",72,1,1,1
1,1,Weng earns $12 an hour for babysitting. Yester...,Weng earns 12/60 = $<<12/60=0.2>>0.2 per minut...,"To find out how much Weng earned, we need to c...","**\nTo find out how much Weng earned, we need ...","['To find out how much Weng earned, we need to...",train,Liam makes $12 per hour tutoring. He tutored f...,"[""First, we need to figure out Liam's earnings...",10,1,1,1
2,2,Betty is saving money for a new wallet which c...,"In the beginning, Betty has only 100 / 2 = $<<...","First, let's calculate how much money Betty ha...","**\nFirst, let's calculate how much money Bett...","[""First, let's calculate how much money Betty ...",train,Oliver is collecting stickers for his album wh...,"['First, we need to figure out how many sticke...",5,1,1,1
3,3,"Julie is reading a 120-page book. Yesterday, s...",Maila read 12 x 2 = <<12*2=24>>24 pages today....,"First, let's find out how many pages Julie has...","**\nFirst, let's find out how many pages Julie...","[""First, let's find out how many pages Julie h...",train,Carlos is painting a fence that has 120 panels...,"['First, we need to calculate the number of pa...",42,1,1,1
4,4,James writes a 3-page letter to 2 different fr...,He writes each friend 3*2=<<3*2=6>>6 pages a w...,"First, let's calculate how many letters James ...","**\nFirst, let's calculate how many letters Ja...","[""First, let's calculate how many letters Jame...",train,Olivia plants 3 flowers in each of 2 different...,"['First, we need to figure out how many times ...",624,1,1,1


In [32]:
df = df.drop(['Score_ExactAnswerInLastSentence', 'Score_LastNumberInString'], axis=1)

In [33]:
df.to_csv('data/113_gsm8k_gpt35_cot_responses_autograded.csv', index=False)

Score Verification

In [50]:
sample_df = df.sample(10)

for i, row in sample_df.iterrows():
    print('#####')
    print(i)
    print()
    print(row['CoT Response'])
    print()
    print('In-Last Sentence: ', row['Score_ExactAnswerInLastSentence'])
    print()
    print('Last Number in String: ', row['Score_LastNumberInString'])
    print()
    print(row['True Answer'])
    print()
    print('#####')

#####
60

First, let's calculate how many tubes of lip gloss Marcy has in total. She has 6 tubs of lip gloss, and each tub holds 2 tubes, so she has 6 * 2 = 12 tubes of lip gloss.

Next, let's calculate how many people each tube of lip gloss can be used for. Each tube of lip gloss can be used for 3 people's makeup.

Finally, let's calculate how many people Marcy can paint with makeup. Marcy has 12 tubes of lip gloss, and each tube can be used for 3 people's makeup, so she can paint 12 * 3 = 36 people with makeup.

Therefore, Marcy can paint makeup for 36 people.

In-Last Sentence:  1

Last Number in String:  1

36

#####
#####
69

First, let's count how many thank you cards Valerie needs to send. She has one for her grandmother, one for her uncle, and one for her aunt. So she needs 3 stamps for the thank you cards.

Next, let's count how many bills Valerie needs to send. She has to send the water bill and the electric bill separately. So she needs 2 stamps for the bills.

Now, let's co

In [49]:
df['Answer'][69]

'Valerie has to send a thank you card to each of 3 people, so she has 3 * 1 = <<3*1=3>>3 thank you cards to mail.\nShe has 2 bills to mail.\nShe has 3 more rebates than bills, so 3 + 2 = <<3+2=5>>5 mail-in rebates to mail.\nShe has twice as many job applications as rebates, so she has 2 * 5 = <<2*5=10>>10 applications to mail.\nShe has 3 + 2 + 5 + 10 = <<3+2+5+10=20>>20 pieces of mail to send.\nThe electric bill needs an extra stamp, so she needs 20 + 1 = <<20+1=21>>21 stamps.\n#### 21'